<a href="https://colab.research.google.com/github/adryduty/computer-vision-cat-project/blob/main/Preprocessing_comp_vision_cat17_June.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np    
from PIL import Image
from google.colab import files, drive
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import time
import matplotlib.patches as patches
import pandas as pd

# Labelling (bounding box)
### The idea is to create a n_cells*n_cells grid on the image to choose the rectangle around Ghiri (you just need to choose the upper left square and the bottom right square).

In [ ]:
def label_maker(upper_left, bottom_right, n_cells):
  '''
  CONSIDER THAT YOU HAVE A n_cells*n_cells MATRIX AND THAT THE PIXELS ARE NORMALIZED TO 1 (they go from 0 to 1)

  This function takes the upper left and the bottom right squares as input and returns:
  - 0 (indicating the class: here is just 0 because we have just Ghiri)
  - X_c (X coordinate of the center of the bounding box)
  - Y_C (Y coordinate of the center of the bounding box)
  - Width of the bounding box
  - Height of the buonding box
  '''
  
  if (bottom_right % n_cells) < (upper_left % n_cells): # To check if bottom_right is (wrong) at the left of upper_left (e.g. if n_cells=10, then upper_left=15 and bottom_right=43 would return an error: (15%10=5) > (43%10=3))
    print("Wrong input: bottom_right is at the left of upper_left")
    return (0,0,0,0,0)
    
  elif (upper_left // n_cells) > (bottom_right // n_cells): # To check if upper_left is (wrong) below bottom_right (e.g. if n_cells=10, then upper_left=43 and bottom_right=15 would return an error (43//10=4)>(15//10=1))
    print("Wrong input: upper_left is below bottom_right")
    return (0,0,0,0,0)

  else:
    num_squares_height = 1 + (bottom_right - upper_left) // n_cells
    height = num_squares_height / n_cells

    num_squares_width = 1 + (bottom_right - upper_left) % n_cells
    width = num_squares_width / n_cells 
    x_a = (upper_left % n_cells + 0.5) / n_cells # upper left x coordinate
    y_a = (upper_left // n_cells + 0.5) / n_cells # upper left y coordinate

    x_b = (bottom_right % n_cells + 0.5) / n_cells # bottom right x coordinate
    y_b = (bottom_right // n_cells + 0.5) / n_cells # bottom right x coordinate

    x_c = (x_a + x_b)/2
    y_c = (y_a + y_b)/2

    return 0, x_c, y_c, width, height 


In [ ]:
def grid_image(google_path_image):
  '''
  This function takes as input the google path of the image you want to apply the grid onto.
  The output is the picture with the squares. 
  '''
  # Open image file
  img = Image.open(google_path_image)
  my_dpi=416.

  img = img.resize(size=(416, 416))

  # Set up figure
  fig=plt.figure(figsize=(float(img.size[0])/my_dpi,float(img.size[1])/my_dpi),dpi=my_dpi)
  ax=fig.add_subplot(111)

  # Remove whitespace from around the image
  fig.subplots_adjust(left=0,right=1,bottom=0,top=1)

  # Set the gridding interval: here we use the major tick interval
  myInterval= 20.8
  loc = plticker.MultipleLocator(base=myInterval)
  ax.xaxis.set_major_locator(loc)
  ax.yaxis.set_major_locator(loc)

  ax.tick_params(axis='x', colors=(0,0,0,0))
  ax.tick_params(axis='y', colors=(0,0,0,0))

  # Remove the borders 
  ax.spines['top'].set_visible(False) 
  ax.spines['right'].set_visible(False) 
  ax.spines['left'].set_visible(False)
  ax.spines['bottom'].set_visible(False)

  # Add the grid
  ax.grid(which='major', axis='both', linestyle='-', linewidth=0.1)

  # Add the image
  ax.imshow(img)

  # Find number of gridsquares in x and y direction
  nx=abs(int(float(ax.get_xlim()[1]-ax.get_xlim()[0])/float(myInterval)))
  ny=abs(int(float(ax.get_ylim()[1]-ax.get_ylim()[0])/float(myInterval)))

  # Add some labels to the gridsquares
  for j in range(ny):
      y=myInterval/2+j * myInterval
      for i in range(nx):
          x=myInterval/2.+float(i) * myInterval
          ax.text(x,y,'{:d}'.format(i+j*nx),color='w',ha='center',va='center', fontsize=1.4)

  plt.show()



In [ ]:
def rectangle_coordinates(x_c, y_c, width, height):
  '''
  This function takes the coordinates of the rectangle's center and the width and the height of a
  rectangle and returns the coordinates of the rectangle's bottom left angle and the width and the height
  multiplied by 416. 
  
  These coordinates will be important to make the rectangle (see the rectangle_maker function)
  '''

  x_bottom_left_angle = (x_c - width/2)*416

  y_bottom_left_angle = (y_c - height/2)*416

  width = width * 416

  height = height * 416

  return x_bottom_left_angle, y_bottom_left_angle, width, height



In [ ]:
def rectangle_maker(google_path_image, x_c, y_c, width, height):

  '''
  This function takes a picture saved in colab, the coordinates of the centers of a rectangle, its width and its height.
  It returns the image with the rectangle around Ghiri and also saved it on colab
  '''

  im = Image.open(google_path_image)
  im = im.resize(size=(416,416))

  # Create figure and axes
  fig, ax = plt.subplots()
  
  x_left_angle, y_left_angle, denorm_width, denorm_height = rectangle_coordinates(x_c, y_c, width, height)
  
  # Create a Rectangle patch
  rect = patches.Rectangle((x_left_angle, y_left_angle), denorm_width, denorm_height, linewidth=1, edgecolor='r', facecolor='none')

  # Add the patch to the Axes
  ax.add_patch(rect)

  ax.tick_params(axis='x', colors=(0,0,0,0))
  ax.tick_params(axis='y', colors=(0,0,0,0))

  # Remove the borders 
  ax.spines['top'].set_visible(False) 
  ax.spines['right'].set_visible(False) 
  ax.spines['left'].set_visible(False)
  ax.spines['bottom'].set_visible(False)

  # Display the image
  ax.imshow(im)

  plt.savefig('/' + google_path_image.split("/")[1]+'/with_rectangle_'+google_path_image.split("/")[2])


### The following chunk allows the user to insert the upper left and the bottom right squares to make the rectangles and so FOR LABELING. The labels will be stored into the **label_vector** list.

#### Each label will be an array **['google_path', 'label', 'x_c', 'y_c', 'width', 'height']**

In [ ]:
def labeler(path_list):
  label_vector = []

  for path in path_list:
    grid_image(path)
    time.sleep(3)
    print(path)
    upper_left = int(input("insert upper left square: "))
    bottom_right = int(input("insert bottom right square: "))
    
    label, x_c, y_c, width, height = label_maker(upper_left, bottom_right, 20)
    if (label, x_c, y_c, width, height) == (0,0,0,0,0):
      print("You have to label again this image:", path)

    label_vector.append([path, label, x_c, y_c, width, height])
    rectangle_maker(path, x_c, y_c, width, height)

  return label_vector

In [ ]:
path_list = []
# for n in range(0,233):
  # path_list.append("/content/img" + str(n) + ".jpg")

In [ ]:
label_vector = labeler(path_list) # Define the path
# Ask the user if it's all right
answer = input("Are all the pictures correct? [YES/NO]")
# If not, then...
if answer.lower() == "no":
  
# Insert the path of the pictures you're not satisfied with
  while True:
    print("Insert the path of those you’re not satisfied with. Write “stop” once you’re done")
    image = input("Insert path: ")
    if image != "stop":
      new_image = labeler([image])
      label_vector.append(new_image[0])
    else:
      break

# There may be some duplicates. Let's drop them.
label_dataframe = pd.DataFrame(label_vector, columns = ['google_path', 'label', 'x_c', 'y_c', 'width', 'height'])
label_dataframe = label_dataframe.drop_duplicates(subset=["google_path"], keep='last', ignore_index=True)
label_dataframe.to_csv('/content/label_dataframe.csv', index = False )